# Persisting memory across Strands Agents sessions

In this example you will learn how to persist memory across different sessions in your Strands Agents. 

We will use the use case of an agent that does web search using a `duckduckgo` search API.

In this notebook, we will:
- Explore the capabilities of a memory-powered Strands agent.
- Learn how to store, retrieve, and list memories.
- Understand how to perform web searches via the agent.
- Interact with the agent in an interactive loop.


### Usage Examples

Storing memories:
```
Remember that I prefer tea over coffee
```

Retrieving memories:
```
What do I prefer to drink?
```

Listing all memories:
```
Show me everything you remember about me
```

### Tips for Memory Usage

- Be explicit when asking the agent to remember information
- Use specific queries to retrieve relevant memories
- Memory persistence enables more natural and contextual conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account and AWS credentials configured in the environment
* Anthropic Claude 3.7 enabled on Amazon Bedrock
* IAM role with permissions to create Amazon Bedrock Knowledge Base, Amazon S3 bucket and Amazon DynamoDB

Let's now install the requirement packages for our Strands Agent

In [1]:
# Install the required packages
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [mem0ai]━━━━ 6/7 [mem0ai]qdrant-client]


In [2]:
# Import Required Libraries
import os
from strands import Agent, tool
from strands_tools import mem0_memory
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException, RatelimitException

## Mem0 Configuration

### Memory Backend Options

The Mem0 Memory Tool supports three different backend configurations:

1. **[OpenSearch](https://aws.amazon.com/opensearch-service/features/serverless/)** (Recommended for AWS environments):
   - Requires AWS credentials and OpenSearch configuration
   - Set `OPENSEARCH_HOST` and optionally `AWS_REGION` (defaults to us-west-2)

2. **[FAISS]((https://faiss.ai/index.html))** (Default for local development):
   - Uses FAISS as the local vector store backend
   - Requires faiss-cpu package for local vector storage
   - No additional configuration needed

3. **Mem0 Platform**:
   - Uses the [Mem0 Platform API](https://docs.mem0.ai/platform/quickstart) for memory management
   - Requires a Mem0 API key : `MEM0_API_KEY` in the environment variables


### Environment Configuration

| Environment Variable | Description | Default | Required For |
|---------------------|-------------|----------|--------------|
| OPENSEARCH_HOST | OpenSearch Serverless Host URL | None | OpenSearch |
| AWS_REGION | AWS Region for OpenSearch | us-west-2 | OpenSearch |
| MEM0_API_KEY | Mem0 Platform API key | None | Mem0 Platform |
| DEV | Enable development mode | false | All modes |


For the scope of this lab, we can use 2 options as a backend for memory management:
### Option 1. [Opensearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) 

This will be our setup architecture for AOSS: 

<div style="text-align:left">
    <img src="images/arch_AOSS.png" width="65%" />
</div>



In [ ]:
# You can manually define your Opensearch Host 
#os.environ["OPENSEARCH_HOST"] = "<your-opensearch-host>.<region>.aoss.amazonaws.com"

In [3]:
# OR - Run the script to Create Opensearch Serverless resource in your AWS Account
!bash prereqs/deploy_OSS.sh

deploying opensearch Serverless ...
Configuration loaded: {'opensearch_collection_name': 'memory-store', 'opensearch_description': 'Memory vector store'}
Creating OpenSearch Serverless collection: memory-store
Waiting for policies to propagate...
Creating collection...

Collection successfully created:
[{'id': 'wu0d64utqlsq9l6v3t14', 'name': 'memory-store', 'status': 'ACTIVE', 'type': 'VECTORSEARCH', 'description': 'Memory vector store', 'arn': 'arn:aws:aoss:us-west-2:210283880577:collection/wu0d64utqlsq9l6v3t14', 'kmsKeyArn': 'auto', 'standbyReplicas': 'ENABLED', 'createdDate': 1752641223932, 'lastModifiedDate': 1752641247022, 'collectionEndpoint': 'https://wu0d64utqlsq9l6v3t14.us-west-2.aoss.amazonaws.com', 'dashboardEndpoint': 'https://wu0d64utqlsq9l6v3t14.us-west-2.aoss.amazonaws.com/_dashboards'}]
Environment variables saved to /workspaces/strands/strands_samples/01-tutorials/01-fundamentals/07-memory-persistent-agents/.env
OpenSearch Serverless collection setup completed!

OPENSE

In [4]:
# Option 1: Opensearch Serverless
from dotenv import load_dotenv
load_dotenv() # take Opensearch environment variables

True


### Option 2 [Mem0 Platform](https://docs.mem0.ai/platform):

#### [NOTE]: This is not Needed if you have already deployed the Opensearch Serverless option.

As an alternative, you can create Mem0 API keys by following the steps [here](https://docs.mem0.ai/platform/quickstart#2-api-key-setup) and add it as an environment variable **MEM0_API_KEY**.
This would be the architecture of the setup for Mem0 Platform:

<div style="text-align:left">
    <img src="images/arch_mem0.png" width="65%" />
</div>


To enable the agent's functionality, we need to configure environment variables for AWS credentials and OpenSearch / Mem0 Platform. These variables are used for memory storage and retrieval.

In [ ]:
# Option 2: Mem0 API key

#os.environ["MEM0_API_KEY"] = "<your-mem0-api-key>"

## Define System Prompt

The `SYSTEM_PROMPT` variable defines the behavior and capabilities of the memory agent. This prompt guides the agent to provide personalized responses based on stored memories and perform web searches when necessary.

In [5]:
# Define a focused system prompt for memory operations
SYSTEM_PROMPT = """You are a helpful personal assistant for a user. Your task is to assist the user by providing personalized responses based on their history. 

Capabilities:
- You can store information using the mem0_memory tool (action="store").
- You can retrieve relevant memories using the mem0_memory tool (action="retrieve").
- You can use duckduckgo_search to find information on the web.

Key Rules:
- Be conversational and natural in your responses.
- Always retrieve memories before responding to the user and use them to inform your response.
- Store any new user information and user preferences in mem0_memory.
- Only share relevant information.
- Politely indicate when you don't have the information.
"""

## Define Web Search Tool

The `websearch` tool using [Duckduckgo Search API](https://github.com/deedy5/duckduckgo_search) function allows the agent to perform web searches. This function handles exceptions and returns search results or appropriate error messages.

In [6]:
@tool
def websearch(
    keywords: str,
    region: str = "us-en",
    max_results: int | None = None,
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DuckDuckGoSearchException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"

## Create Memory Agent

We will now initialize the memory-focused agent using the defined tools and system prompt. The Strands agent is capable of:
1. Storing and retrieving memories based on context. It uses memory to create more personalized and contextual AI interactions.
2. Performing web searches using DuckDuckGo to give updated information.

In [14]:
# Create an agent with memory, websearch tool
USER_ID = "new_user" # Replace with actual user ID

memory_agent = Agent(
    system_prompt=SYSTEM_PROMPT,
    tools=[mem0_memory, websearch],
)

## Demonstrate Memory Operations

The following examples demonstrate how to store, retrieve, and list memories using the memory agent.

- **store**: Save important information for later retrieval
  - Store user preferences
  - Remember important facts
  - Maintain conversation context

- **retrieve**: Access relevant memories based on queries
  - Find previously stored information
  - Provide personalized responses based on user history

- **list**: View all stored memories
  - See what information has been retained
  - Audit stored memories

In [16]:
# Store initial memories to demonstrate retrieval
memory_agent.tool.mem0_memory(
    action="store", content=f"The user's name is {USER_ID}.", user_id=USER_ID
)
memory_agent.tool.mem0_memory(
    action="store", 
    content="I like to drink tea more than coffee.", 
    user_id=USER_ID
)

╭─────────────────────────────────────────── Memory for user new_user ────────────────────────────────────────────╮
│ The user's name is new_user.                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Memory for user new_user ────────────────────────────────────────────╮
│ I like to drink tea more than coffee.                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_mem0_memory_591696000',
 'status': 'success',
 'content': [{'text': '[]'}]}

In [17]:
# Retrieve memories
retrieved_memories = memory_agent.tool.mem0_memory(
    action="retrieve", query="What is the user's name?", user_id=USER_ID
)
print("Retrieved Memories:", retrieved_memories)

╭──────────────────────────────────────────────── Search Results ─────────────────────────────────────────────────╮
│                                                 Search Results                                                  │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃           ┃                                                    ┃          ┃ Created   ┃          ┃          ┃ │
│ ┃ ID        ┃ Memory                                             ┃ Relevan… ┃ At        ┃ User ID  ┃ Metadata ┃ │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ 533a63a6… │ Name is new_user                                   │ 0.67664… │ 2025-07-… │ new_user │ None     │ │
│ │ 0da6c310… │ Interested in events happening in New York City    │ 0.55103… │ 2025-07-… │ new_user │ None     │ │
│ │ d4f59208… │ Prefers tea over coffee                            │ 0.54533… │ 2025-07-… │ new_user │ None     │ │
│ └───────────┴────────────────────────────────────────────────────┴──────────┴───────────┴──────────┴──────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Retrieved Memories: {'toolUseId': 'tooluse_mem0_memory_253128646', 'status': 'success', 'content': [{'text': '[\n  {\n    "id": "533a63a6-389b-43c4-8ad8-1bbfa69af493",\n    "memory": "Name is new_user",\n    "hash": "8fe736239b1e08f5db6bdb05dd4aa9c3",\n    "metadata": null,\n    "score": 0.67664516,\n    "created_at": "2025-07-15T21:49:59.663804-07:00",\n    "updated_at": null,\n    "user_id": "new_user"\n  },\n  {\n    "id": "0da6c310-dd96-4fe4-b654-6c684867757c",\n    "memory": "Interested in events happening in New York City",\n    "hash": "09e84ef882e392a99aa1df28e1021591",\n    "metadata": null,\n    "score": 0.5510382,\n    "created_at": "2025-07-15T21:52:33.173157-07:00",\n    "updated_at": null,\n    "user_id": "new_user"\n  },\n  {\n    "id": "d4f59208-9d8d-461f-8d85-cecdbf842839",\n    "memory": "Prefers tea over coffee",\n    "hash": "5b1df6e791a3f297da049affce574059",\n    "metadata": null,\n    "score": 0.5453303,\n    "created_at": "2025-07-15T21:50:10.907485-07:00",\n   

In [18]:
# Retrieve memories about preferences
memory_agent.tool.mem0_memory(
    action="retrieve",
    query="What are the my drink preferences?",
    user_id=USER_ID
)

╭──────────────────────────────────────────────── Search Results ─────────────────────────────────────────────────╮
│                                                 Search Results                                                  │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃           ┃                                                    ┃          ┃ Created   ┃          ┃          ┃ │
│ ┃ ID        ┃ Memory                                             ┃ Relevan… ┃ At        ┃ User ID  ┃ Metadata ┃ │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ d4f59208… │ Prefers tea over coffee                            │ 0.65325… │ 2025-07-… │ new_user │ None     │ │
│ │ 0da6c310… │ Interested in events happening in New York City    │ 0.54864… │ 2025-07-… │ new_user │ None     │ │
│ │ 533a63a6… │ Name is new_user                                   │ 0.53423… │ 2025-07-… │ new_user │ None     │ │
│ └───────────┴────────────────────────────────────────────────────┴──────────┴───────────┴──────────┴──────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

{'toolUseId': 'tooluse_mem0_memory_340238631',
 'status': 'success',
 'content': [{'text': '[\n  {\n    "id": "d4f59208-9d8d-461f-8d85-cecdbf842839",\n    "memory": "Prefers tea over coffee",\n    "hash": "5b1df6e791a3f297da049affce574059",\n    "metadata": null,\n    "score": 0.6532587,\n    "created_at": "2025-07-15T21:50:10.907485-07:00",\n    "updated_at": null,\n    "user_id": "new_user"\n  },\n  {\n    "id": "0da6c310-dd96-4fe4-b654-6c684867757c",\n    "memory": "Interested in events happening in New York City",\n    "hash": "09e84ef882e392a99aa1df28e1021591",\n    "metadata": null,\n    "score": 0.54864514,\n    "created_at": "2025-07-15T21:52:33.173157-07:00",\n    "updated_at": null,\n    "user_id": "new_user"\n  },\n  {\n    "id": "533a63a6-389b-43c4-8ad8-1bbfa69af493",\n    "memory": "Name is new_user",\n    "hash": "8fe736239b1e08f5db6bdb05dd4aa9c3",\n    "metadata": null,\n    "score": 0.53423786,\n    "created_at": "2025-07-15T21:49:59.663804-07:00",\n    "updated_at": nu

In [19]:
# Ask the agent a question
response = memory_agent("What are the events happening in the New York today?")
print(response)


Tool #1: mem0_memory


╭──────────────────────────────────────────────── Search Results ─────────────────────────────────────────────────╮
│                                                 Search Results                                                  │
│ ┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃           ┃                                                    ┃          ┃ Created   ┃          ┃          ┃ │
│ ┃ ID        ┃ Memory                                             ┃ Relevan… ┃ At        ┃ User ID  ┃ Metadata ┃ │
│ ┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ 0da6c310… │ Interested in events happening in New York City    │ 0.86235… │ 2025-07-… │ new_user │ None     │ │
│ │ d4f59208… │ Prefers tea over coffee                            │ 0.56447  │ 2025-07-… │ new_user │ None     │ │
│ │ 533a63a6… │ Name is new_user                                   │ 0.52974… │ 2025-07-… │ new_user │ None     │ │
│ └───────────┴────────────────────────────────────────────────────┴──────────┴───────────┴──────────┴──────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Perfect! I can see you're interested in New York City events. Let me search for what's happening today in NYC.
Tool #2: websearch


/tmp/ipykernel_60372/3079496950.py:16: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(keywords, region=region, max_results=max_results)


Let me get more specific information about today's events in NYC:
Tool #3: websearch


/tmp/ipykernel_60372/3079496950.py:16: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(keywords, region=region, max_results=max_results)


Let me search more specifically for NYC events happening today:
Tool #4: websearch


/tmp/ipykernel_60372/3079496950.py:16: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  results = DDGS().text(keywords, region=region, max_results=max_results)


Based on your interest in New York City events, here's what I found happening today and this week in NYC:

## Events Happening in New York City Today

Since you're interested in NYC events, here are some great options for today:

**General Event Categories Available:**
- **Broadway Shows** - Multiple shows running daily
- **Live Music & Concerts** - Various venues hosting performances
- **Art Exhibitions** - Museums and galleries with ongoing displays
- **Comedy Shows** - Stand-up performances at comedy clubs
- **Free Events** - Many free activities throughout the city

**Popular January 2025 Highlights:**
- **Winter Activities** - Ice skating at Central Park and Rockefeller Center
- **Museum Exhibitions** - World-class art and cultural displays
- **Live Music Venues** - Concert halls and clubs with daily performances
- **Theater District** - Broadway and off-Broadway productions

For the most up-to-date specific events happening today, I'd recommend checking:
- **Time Out New York** f

In [20]:

# List all stored memories
print("All Stored Memories:")
all_memories = memory_agent.tool.mem0_memory(
    action="list", user_id=USER_ID
)

All Stored Memories:


╭───────────────────────────────────────────────── Memories List ─────────────────────────────────────────────────╮
│                                                    Memories                                                     │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓ │
│ ┃ ID             ┃ Memory                                             ┃ Created At      ┃ User ID  ┃ Metadata ┃ │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩ │
│ │ 533a63a6-389b… │ Name is new_user                                   │ 2025-07-15T21:… │ new_user │ None     │ │
│ │ d4f59208-9d8d… │ Prefers tea over coffee                            │ 2025-07-15T21:… │ new_user │ None     │ │
│ │ 0da6c310-dd96… │ Interested in events happening in New York City    │ 2025-07-15T21:… │ new_user │ None     │ │
│ └────────────────┴────────────────────────────────────────────────────┴─────────────────┴──────────┴──────────┘ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Interactive Agent Usage

Finally, we provide an interactive loop for users to interact with the memory agent. Users can store new memories, retrieve existing ones, or list all stored memories.

To test interactive usage: 
1. Install the requirements: `pip install -r requirements.txt`
1. Run the python file `personal_agent_with_memory.py`. 

## Conclusion

This notebook demonstrates how to create a personal agent with memory capabilities using the Strands framework. The agent can:

1. Store information about the user
2. Retrieve relevant memories based on context
3. Search the web for additional information
4. Provide personalized responses

By combining these capabilities, the agent can maintain context across conversations and provide more personalized assistance over time.

### Cleanup

Run this bash script to clean up the Opensearch Serverless resources. You don't need to run this if you used the "MEM0_PLATFORM_API".

In [21]:
!sh prereqs/cleanup_OSS.sh

Removing Opensearch Serverless resources ...
Configuration loaded: {'opensearch_collection_name': 'memory-store', 'opensearch_description': 'Memory vector store'}
Deleting OpenSearch Serverless collection: memory-store
Found collection ID: wu0d64utqlsq9l6v3t14
Collection memory-store deleted successfully!
Waiting for collection deletion to complete...
Access policy memory-store-ap-1752641217 deleted successfully!
Network policy memory-store-np-1752641217 deleted successfully!
Encryption policy memory-store-sp-1752641217 deleted successfully!
OpenSearch Serverless resources cleanup completed!
Deleted SSM parameter: memory-store-hostname
Deleted SSM parameter: memory-store-id
Deleted SSM parameter: memory-store-arn
Deleted SSM parameter: memory-store-name
Deleted SSM parameter: memory-store-encryption_policy
Deleted SSM parameter: memory-store-network_policy
Deleted SSM parameter: memory-store-access_policy
Removed .env file: /workspaces/strands/strands_samples/01-tutorials/01-fundamenta